In [39]:
import re

In [7]:
#знакомьтесь франкенштейн

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name_food = 'numblilbug/food-rubert-sentiment-model'
model_name_interior = 'numblilbug/interior-rubert-sentiment-model'
model_name_price = 'numblilbug/price-rubert-sentiment-model'
model_name_service = 'numblilbug/service-rubert-sentiment-model'
model_name_whole = 'numblilbug/service-rubert-sentiment-model'

food_tokenizer = AutoTokenizer.from_pretrained(model_name_food)
food_model = AutoModelForSequenceClassification.from_pretrained(model_name_food)

interior_tokenizer = AutoTokenizer.from_pretrained(model_name_interior)
interior_model = AutoModelForSequenceClassification.from_pretrained(model_name_interior)

price_tokenizer = AutoTokenizer.from_pretrained(model_name_price)
price_model = AutoModelForSequenceClassification.from_pretrained(model_name_price)

service_tokenizer = AutoTokenizer.from_pretrained(model_name_service)
service_model = AutoModelForSequenceClassification.from_pretrained(model_name_service)

whole_tokenizer = AutoTokenizer.from_pretrained(model_name_whole)
whole_model = AutoModelForSequenceClassification.from_pretrained(model_name_whole)


In [40]:
file_path = '/content/dev_reviews (1).txt'


with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()
    reviews_list = [line.split(maxsplit=1)[1].strip() for line in file.readlines()]
numbers = re.findall(r'\b\d+\b', file_content)


In [30]:
from transformers import pipeline
import torch

def predict_aspect(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.sigmoid(logits)
    predicted_class = torch.argmax(probabilities, dim=1)
    return predicted_class.item()


food_predictions = [predict_aspect(text, food_tokenizer, food_model) for text in reviews_list]


In [32]:
interior_predictions = [predict_aspect(text, interior_tokenizer, interior_model) for text in reviews_list]
price_predictions = [predict_aspect(text, price_tokenizer, price_model) for text in reviews_list]
service_predictions = [predict_aspect(text, service_tokenizer, service_model) for text in reviews_list]
whole_predictions = [predict_aspect(text, whole_tokenizer, whole_model) for text in reviews_list]


In [46]:
file_path = '/content/dev_cats (1).txt'

sentiment_mapping = {'positive': 0, 'negative': 1, 'both': 2, 'neutral': 3, 'absence': 4}


food_sentiments = []
interior_sentiments = []
price_sentiments = []
whole_sentiments = []
service_sentiments = []


with open(file_path, 'r') as file:
    for line in file:
        if not line.strip():
            continue

        parts = line.strip().split('\t')
        aspect = parts[1]
        sentiment_word = parts[2]


        sentiment_number = sentiment_mapping.get(sentiment_word, -1)

        if aspect == 'Food':
            food_sentiments.append(sentiment_number)
        elif aspect == 'Interior':
            interior_sentiments.append(sentiment_number)
        elif aspect == 'Price':
            price_sentiments.append(sentiment_number)
        elif aspect == 'Whole':
            whole_sentiments.append(sentiment_number)
        elif aspect == 'Service':
            service_sentiments.append(sentiment_number)



In [47]:
accuracy_dict = {}


def calculate_accuracy(actual, predicted):
    correct = sum(1 for a, p in zip(actual, predicted) if a == p)
    total = len(actual)
    return correct / total

accuracy_dict['Food'] = calculate_accuracy(food_sentiments, food_predictions)
accuracy_dict['Interior'] = calculate_accuracy(interior_sentiments, interior_predictions)
accuracy_dict['Price'] = calculate_accuracy(price_sentiments, price_predictions)
accuracy_dict['Whole'] = calculate_accuracy(whole_sentiments, whole_predictions)
accuracy_dict['Service'] = calculate_accuracy(service_sentiments, service_predictions)

In [48]:
all_actual_sentiments = food_sentiments + interior_sentiments + price_sentiments + whole_sentiments + service_sentiments
all_predicted_sentiments = food_predictions + interior_predictions + price_predictions + whole_predictions + service_predictions

overall_accuracy = calculate_accuracy(all_actual_sentiments, all_predicted_sentiments)

In [49]:
overall_accuracy

0.7014084507042253